In [53]:
import nltk
from nltk.stem import PorterStemmer
from textblob import TextBlob
from nltk.corpus import wordnet as w,stopwords 
import json
import numpy as np
import os
import re
import gensim

In [55]:
#loading word2vec model
model=gensim.models.Word2Vec.load('../Word2Vec_model/model')

In [151]:
#fetching aspects
aspects={}
for root, dirs, files in os.walk('../Aspects/'):
        for name in files:
            with open('../Aspects/'+name) as f:
                aspects[name]=json.load(f) 

In [84]:
#initialising porter stemmer
ps=PorterStemmer()

In [16]:
#fetch all reviews
data=[]
for root, dirs, files in os.walk('../Reviews/'):
    for name in dirs:                             #all review folders
        with open('../Reviews/'+name+'/review.txt') as f:
            data=data+json.load(f)
            f.close()
dataset=[]
dataset=np.asarray(data)
dataset=np.concatenate(dataset)               #contains all the reviews

In [108]:
def remove_stopwords(tokens):                                # for removing stopwords
    stop=set(stopwords.words('english'))
    stopwords_tokens=[token for token in tokens if token not in stop]
    return stopwords_tokens

In [118]:
def analyse_aspect(tokens):            # analyses to which aspect a set of tokens of a sentence belong to
    aspect_flags={}
    for aspect_name in files:
        aspect_flags[aspect_name]=0

    for token in tokens:
        temp_token=ps.stem(token)
        for aspect_name in files:
            for aspect in aspects[aspect_name]:
                temp_aspect=ps.stem(aspect)
                if(temp_token == temp_aspect):
                    aspect_flags[aspect_name]=aspect_flags[aspect_name]+1
    return aspect_flags

In [157]:
def analyse_sentiment(aspect_flags,sentence):    #analyses the sentiment and stores the sentence to the specified class
    temp=TextBlob(sentence)
    polarity=temp.sentiment[0]            
    if(polarity>=0 and polarity<=0.25):
        file_name='neg'
    elif(polarity>0.25 and polarity<=0.45):
        file_name='sli_neg'
    elif(polarity>0.55 and polarity<=0.75):
        file_name='sli_pos'
    elif(polarity>0.75 and polarity<=1):
        file_name='pos' 
    else :
        file_name='neutral'
    print 'sentence :'+ sentence
    print 'polarity :'+ file_name
    
    gen_flag=0
    for aspect_name in files:                 # for all aspect names
        if(aspect_flags[aspect_name]>0):
            print 'aspect name :' +aspect_name
            gen_flag=1             # for general category
            file_ptr=open('../Dataset/'+aspect_name+'/'+file_name,'r')
            temp_data=json.load(file_ptr)
            file_ptr.close()
            file_ptr=open('../Dataset/'+aspect_name+'/'+file_name,'w')
            temp_data.append(sentence)
            json.dump(temp_data,file_ptr)
            file_ptr.close() 
    
    if(gen_flag==0):
        print 'Aspect Name : general category'
        file_ptr=open('../Dataset/Aspect7/'+file_name,'r')
        temp_data=json.load(file_ptr)
        file_ptr.close()
        file_ptr=open('../Dataset/Aspect7/'+file_name,'w')
        temp_data.append(sentence)
        json.dump(temp_data,file_ptr)
        file_ptr.close()     

In [111]:
def tokenize_sent(sentence):                         # for tokenizing single sentence
    temp_tokens=nltk.word_tokenize(sentence)
    sent_tokens=remove_stopwords(temp_tokens)         # to be reconsidered
    aspect_flags=analyse_aspect(sent_tokens)
    analyse_sentiment(aspect_flags,sentence)

In [112]:
def tokenize_review(review):                # for tokenizing single reviews
    review=review.lower()
    review=TextBlob(review)
    review=str(review.correct())           #to be corrected
    review=review.replace('.',',')        
    sentences=re.split('and | but |, ',review)        #splitting criteria
    for sentence in sentences:
        tokenize_sent(sentence)

In [113]:
def tokenize_all(dataset):               # for tokenizing all the reviews
    print dataset
    for review in dataset:
        tokenize_review(review)

In [153]:
tokenize_all(dataset[100:105])

[ u' IBM is a dynamic organisation &amp; try to keep up pace with fast changing trends in industry. Employees are  provided platform &amp; space to express ideas freely '
 u' Best Company to work with. Learned a lot of technologies, Management is very friendly and workplace culture is the one which makes you comfortable. The most enjoyable part is to explore your knowledge in sevaral domains and technology. '
 u' There is a huge pool of talent in IBM but I would say one has to put right eggs in the right basket. People who carry specific skill should be optimally utilized given their skill set. '
 u' IBM is a vast organisation. So every project or assignment is as good as a company change. Some projects can turn out to be a very good learning experience, but some may not. Good thing is you can change your projects every 2 years. '
 u' Corporate culture Good place to work Management is not up to the mark Hardest part of the job is nothing new to learn and scope for my work Enjoyable par

In [154]:
#clear all files
def clear_all():
    temp_files=files
    temp_files.append('Aspect7')
    for file_name in files:
        with open('../Dataset/'+file_name+'/pos','w') as f:
            json.dump([],f)
        with open('../Dataset/'+file_name+'/neg','w') as f:
            json.dump([],f)
        with open('../Dataset/'+file_name+'/sli_pos','w') as f:
            json.dump([],f)
        with open('../Dataset/'+file_name+'/sli_neg','w') as f:
            json.dump([],f)
        with open('../Dataset/'+file_name+'/neutral','w') as f:
            json.dump([],f)        

In [155]:
clear_all()